# Data Preparation

In [ ]:
from __future__ import absolute_import
from numpy import asarray
from numpy import save
from mqt.predictor.driver import Predictor
from mqt.predictor import utils

import numpy as np

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

training_data, names_list, scores_list = utils.load_training_data()
X, y = zip(*training_data)
X = list(X)
y = list(y)
for i in range(len(X)):
    X[i] = list(X[i])
    scores_list[i] = list(scores_list[i])


X, y, indices = np.array(X), np.array(y), np.array(range(len(y)))

non_zero_indices = []
for i in range(len(X[0])):
    if sum(X[:, i]) > 0:
        non_zero_indices.append(i)
X = X[:, non_zero_indices]
data = asarray(non_zero_indices)
save("non_zero_indices.npy", data)

(
    X_train,
    X_test,
    y_train,
    y_test,
    indices_train,
    indices_test,
) = train_test_split(X, y, indices, test_size=0.3, random_state=5)

scores_filtered = [scores_list[i] for i in indices_test]
names_filtered = [names_list[i] for i in indices_test]

predictor = Predictor()
performance = []

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier


clf = RandomForestClassifier(random_state=0)

tree_param = [
    {
        "n_estimators": [100, 200, 500],
        "max_features": ["auto", "sqrt"],
        "max_depth": list(range(8, 30, 6)),
        "min_samples_split": list(range(2, 20, 6)),
        "min_samples_leaf": list(range(2, 20, 6)),
        "bootstrap": [True, False],
    },
]

clf = GridSearchCV(clf, tree_param, cv=5, n_jobs=8).fit(X_train, y_train)


y_pred = np.array(list(clf.predict(X_test)))
res = predictor.plot_eval_histogram(
    scores_filtered, y_pred, y_test, filename="RandomForestClassifier"
)

print("Best Accuracy: ", clf.best_score_)
top3 = (res.count(1) + res.count(2) + res.count(3)) / len(res)
print("Top 3: ", top3)
print("Feature Importance: ", clf.best_estimator_.feature_importances_)
performance.append(("Random Forest", clf.best_score_, top3, max(res)))

predictor.set_classifier(clf.best_estimator_)
utils.save_classifier(clf.best_estimator_)

### Feature Importances

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

openqasm_qc_list = utils.get_openqasm_gates()
feature_names = [openqasm_qc_list[i] for i in range(0, len(openqasm_qc_list))]
feature_names.append("num_qubits")
feature_names.append("depth")
feature_names.append("program_communication")
feature_names.append("critical_depth")
feature_names.append("entanglement_ratio")
feature_names.append("parallelism")
feature_names.append("liveness")
feature_names = [feature_names[i] for i in non_zero_indices]

importances = clf.best_estimator_.feature_importances_
std = np.std(
    [tree.feature_importances_ for tree in clf.best_estimator_.estimators_], axis=0
)

idx = np.argsort(-importances)

plt.figure(figsize=(8, 8))
plt.bar(np.array(feature_names)[idx], np.array(importances)[idx])
plt.errorbar(
    np.array(feature_names)[idx],
    np.array(importances)[idx],
    np.array(std)[idx],
    fmt="o",
    color="lightgreen",
)
plt.xticks(rotation=90, fontsize=18)
plt.yticks(fontsize=18)
plt.ylabel("Relative feature importance", fontsize=18)
plt.tight_layout()
plt.savefig("results/feature_importances.pdf")
plt.show()

In [ ]:
predictor.plot_eval_all_detailed_compact_normed(
    names_filtered, scores_filtered, y_pred, y_test
)

# GradientBoostingClassifier

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

clf = GradientBoostingClassifier()

param_grid = {
    "learning_rate": [0.01, 0.1, 1],
}

clf = GridSearchCV(clf, param_grid, cv=5, n_jobs=8).fit(X_train, y_train)

y_pred = np.array(list(clf.predict(X_test)))
res = predictor.plot_eval_histogram(
    scores_filtered, y_pred, y_test, filename="GradientBoostingClassifier"
)

print("Best Accuracy: ", clf.best_score_)
top3 = (res.count(1) + res.count(2) + res.count(3)) / len(res)
print("Top 3: ", top3)
performance.append(("Gradient Boosting", clf.best_score_, top3, max(res)))

# Current Decision Tree Classifier

In [ ]:
# predictor.train_decision_tree_classifier(X, y, names_list, scores_list);

In [ ]:
from sklearn import tree
from sklearn.model_selection import GridSearchCV

clf = tree.DecisionTreeClassifier(random_state=5)

tree_param = [
    {
        "criterion": ["entropy", "gini"],
        "max_depth": list(range(1, 15, 1)),
        "min_samples_split": list(range(2, 20, 4)),
        "min_samples_leaf": list(range(2, 20, 4)),
        "max_leaf_nodes": list(range(2, 200, 40)),
        "max_features": list(range(1, len(non_zero_indices), 10)),
    },
]
clf = GridSearchCV(clf, tree_param, cv=5, n_jobs=8).fit(X_train, y_train)
y_pred = np.array(list(clf.predict(X_test)))
res = predictor.plot_eval_histogram(
    scores_filtered, y_pred, y_test, filename="DecisionTreeClassifier"
)

print("Best Accuracy: ", clf.best_score_)
top3 = (res.count(1) + res.count(2) + res.count(3)) / len(res)
print("Top 3: ", top3)
print("Feature Importance: ", clf.best_estimator_.feature_importances_)
performance.append(("Decision Tree", clf.best_score_, top3, max(res)))

# Nearest Neighbor

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

clf = KNeighborsClassifier()
param_grid = dict(n_neighbors=range(1, 10, 1))
clf = GridSearchCV(clf, param_grid, cv=5, n_jobs=8).fit(X_train, y_train)

y_pred = np.array(list(clf.predict(X_test)))
res = predictor.plot_eval_histogram(
    scores_filtered, y_pred, y_test, filename="KNeighborsClassifier"
)

print("Best Accuracy: ", clf.best_score_)
top3 = (res.count(1) + res.count(2) + res.count(3)) / len(res)
print(("Top 3: ", top3))
performance.append(("Nearest Neighbor", clf.best_score_, top3, max(res)))

# MLPClassifier

In [ ]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier( max_iter=1000)

param_grid = {
    'hidden_layer_sizes': [(50,50,50), (50,100,50), (100,)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
}

clf = GridSearchCV(clf, param_grid, cv=5, n_jobs=8).fit(X_train, y_train)

y_pred = np.array(list(clf.predict(X_test)))
res = predictor.plot_eval_histogram(
    scores_filtered, y_pred, y_test, filename="MLPClassifier"
)

print("Best Accuracy: ", clf.best_score_)
top3 = (res.count(1) + res.count(2) + res.count(3)) / len(res)
print("Top 3: ", top3)
performance.append(("Multilayer Perceptron", clf.best_score_, top3, max(res)))

# SVM

In [ ]:
from sklearn import svm

clf = svm.SVC()
param_grid = {"C": [0.1, 1, 10], "gamma": [1, 0.1, 0.01], "kernel": ["rbf", "sigmoid"]}
clf = GridSearchCV(clf, param_grid, cv=5, n_jobs=8).fit(X_train, y_train)

y_pred = np.array(list(clf.predict(X_test)))
res = predictor.plot_eval_histogram(scores_filtered, y_pred, y_test, filename="SVM")

print("Best Accuracy: ", clf.best_score_)
top3 = (res.count(1) + res.count(2) + res.count(3)) / len(res)
print("Top 3: ", top3)
performance.append(("Support Vector Machine", clf.best_score_, top3, max(res)))

# Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB

clf = GaussianNB()
param_grid = {"var_smoothing": np.logspace(0, -9, num=100)}
clf = GridSearchCV(clf, param_grid, cv=5, n_jobs=8).fit(X_train, y_train)

y_pred = np.array(list(clf.predict(X_test)))
res = predictor.plot_eval_histogram(
    scores_filtered, y_pred, y_test, filename="GaussianNB"
)

print("Best Accuracy: ", clf.best_score_)
top3 = (res.count(1) + res.count(2) + res.count(3)) / len(res)
print("Top 3: ", top3)
performance.append(("Naive Bayes", clf.best_score_, top3, max(res)))

# Save Performance Results

In [ ]:
print(performance)

filename = "results/performances.csv"
with open(filename, "w") as f:
    f.write("Algorithm, Accuracy, Top3, Worst Rank\n")
    for sublist in performance:
        line = "{}, {}, {}, {} \n".format(
            sublist[0], sublist[1], sublist[2], sublist[3]
        )
        f.write(line)

# Run-Time Comparison

In [8]:
target_path = "/Users/nils/Desktop/runtime_comp/"
from mqt.predictor.driver import Predictor
from mqt.predictor import utils
import numpy as np
from mqt.bench import benchmark_generator
import time

pred = Predictor()
filename="dj_indep_qiskit_7.qasm"

results =[]
for _ in range(30):

    start = time.time()
    pred.compile_all_circuits_for_qc(filename = filename, target_path=target_path, timeout=10000)


    start_postprocess = time.time()
    utils.postprocess_ocr_qasm_files(target_path)
    end_postprocess = time.time()
    diff_postprocess = (end_postprocess - start_postprocess)

    res = pred.generate_training_sample(file = filename, target_path=target_path)
    end = time.time()

    diff_all_compile = (end - start)
    results.append(diff_all_compile-diff_postprocess)
print(results)
print(np.mean(results))

compile_all_circuits_for_qc: dj_indep_qiskit_7.qasm
New qasm file for:  /Users/nils/Desktop/runtime_comp/dj_indep_qiskit_8_26.qasm
New qasm file for:  /Users/nils/Desktop/runtime_comp/dj_indep_qiskit_7_28.qasm
New qasm file for:  /Users/nils/Desktop/runtime_comp/dj_indep_qiskit_7_24.qasm
New qasm file for:  /Users/nils/Desktop/runtime_comp/dj_indep_qiskit_7_25.qasm
New qasm file for:  /Users/nils/Desktop/runtime_comp/dj_indep_qiskit_7_29.qasm
New qasm file for:  /Users/nils/Desktop/runtime_comp/dj_indep_qiskit_7_26.qasm
New qasm file for:  /Users/nils/Desktop/runtime_comp/dj_indep_qiskit_7_27.qasm
New qasm file for:  /Users/nils/Desktop/runtime_comp/dj_indep_qiskit_8_29.qasm
New qasm file for:  /Users/nils/Desktop/runtime_comp/dj_indep_qiskit_8_25.qasm
New qasm file for:  /Users/nils/Desktop/runtime_comp/dj_indep_qiskit_8_24.qasm
New qasm file for:  /Users/nils/Desktop/runtime_comp/dj_indep_qiskit_8_28.qasm
Checking  dj_indep_qiskit_7.qasm
compile_all_circuits_for_qc: dj_indep_qiskit_7

compile_all_circuits_for_qc: dj_indep_qiskit_7.qasm
New qasm file for:  /Users/nils/Desktop/runtime_comp/dj_indep_qiskit_8_26.qasm
New qasm file for:  /Users/nils/Desktop/runtime_comp/dj_indep_qiskit_7_28.qasm
New qasm file for:  /Users/nils/Desktop/runtime_comp/dj_indep_qiskit_7_24.qasm
New qasm file for:  /Users/nils/Desktop/runtime_comp/dj_indep_qiskit_7_25.qasm
New qasm file for:  /Users/nils/Desktop/runtime_comp/dj_indep_qiskit_7_29.qasm
New qasm file for:  /Users/nils/Desktop/runtime_comp/dj_indep_qiskit_7_26.qasm
New qasm file for:  /Users/nils/Desktop/runtime_comp/dj_indep_qiskit_7_27.qasm
New qasm file for:  /Users/nils/Desktop/runtime_comp/dj_indep_qiskit_8_29.qasm
New qasm file for:  /Users/nils/Desktop/runtime_comp/dj_indep_qiskit_8_25.qasm
New qasm file for:  /Users/nils/Desktop/runtime_comp/dj_indep_qiskit_8_24.qasm
New qasm file for:  /Users/nils/Desktop/runtime_comp/dj_indep_qiskit_8_28.qasm
Checking  dj_indep_qiskit_7.qasm
compile_all_circuits_for_qc: dj_indep_qiskit_7

compile_all_circuits_for_qc: dj_indep_qiskit_7.qasm
New qasm file for:  /Users/nils/Desktop/runtime_comp/dj_indep_qiskit_8_26.qasm
New qasm file for:  /Users/nils/Desktop/runtime_comp/dj_indep_qiskit_7_28.qasm
New qasm file for:  /Users/nils/Desktop/runtime_comp/dj_indep_qiskit_7_24.qasm
New qasm file for:  /Users/nils/Desktop/runtime_comp/dj_indep_qiskit_7_25.qasm
New qasm file for:  /Users/nils/Desktop/runtime_comp/dj_indep_qiskit_7_29.qasm
New qasm file for:  /Users/nils/Desktop/runtime_comp/dj_indep_qiskit_7_26.qasm
New qasm file for:  /Users/nils/Desktop/runtime_comp/dj_indep_qiskit_7_27.qasm
New qasm file for:  /Users/nils/Desktop/runtime_comp/dj_indep_qiskit_8_29.qasm
New qasm file for:  /Users/nils/Desktop/runtime_comp/dj_indep_qiskit_8_25.qasm
New qasm file for:  /Users/nils/Desktop/runtime_comp/dj_indep_qiskit_8_24.qasm
New qasm file for:  /Users/nils/Desktop/runtime_comp/dj_indep_qiskit_8_28.qasm
Checking  dj_indep_qiskit_7.qasm
compile_all_circuits_for_qc: dj_indep_qiskit_7

compile_all_circuits_for_qc: dj_indep_qiskit_7.qasm
New qasm file for:  /Users/nils/Desktop/runtime_comp/dj_indep_qiskit_8_26.qasm
New qasm file for:  /Users/nils/Desktop/runtime_comp/dj_indep_qiskit_7_28.qasm
New qasm file for:  /Users/nils/Desktop/runtime_comp/dj_indep_qiskit_7_24.qasm
New qasm file for:  /Users/nils/Desktop/runtime_comp/dj_indep_qiskit_7_25.qasm
New qasm file for:  /Users/nils/Desktop/runtime_comp/dj_indep_qiskit_7_29.qasm
New qasm file for:  /Users/nils/Desktop/runtime_comp/dj_indep_qiskit_7_26.qasm
New qasm file for:  /Users/nils/Desktop/runtime_comp/dj_indep_qiskit_7_27.qasm
New qasm file for:  /Users/nils/Desktop/runtime_comp/dj_indep_qiskit_8_29.qasm
New qasm file for:  /Users/nils/Desktop/runtime_comp/dj_indep_qiskit_8_25.qasm
New qasm file for:  /Users/nils/Desktop/runtime_comp/dj_indep_qiskit_8_24.qasm
New qasm file for:  /Users/nils/Desktop/runtime_comp/dj_indep_qiskit_8_28.qasm
Checking  dj_indep_qiskit_7.qasm
compile_all_circuits_for_qc: dj_indep_qiskit_7

In [7]:
results =[]
for _ in range(30):
    start = time.time()
    prediction = pred.predict("training_samples/" + filename)
    pred.compile_predicted_compilation_path("training_samples/" + filename, prediction)
    end = time.time()

    diff_predictor = round((end - start),3)
    results.append(diff_predictor)
print(results)
print(np.mean(results))

Reading from .qasm path:  training_samples/dj_indep_qiskit_7.qasm

Reading from .qasm path:  training_samples/dj_indep_qiskit_7.qasm

Reading from .qasm path:  training_samples/dj_indep_qiskit_7.qasm

Reading from .qasm path:  training_samples/dj_indep_qiskit_7.qasm

Reading from .qasm path:  training_samples/dj_indep_qiskit_7.qasm

Reading from .qasm path:  training_samples/dj_indep_qiskit_7.qasm

Reading from .qasm path:  training_samples/dj_indep_qiskit_7.qasm

Reading from .qasm path:  training_samples/dj_indep_qiskit_7.qasm

Reading from .qasm path:  training_samples/dj_indep_qiskit_7.qasm

Reading from .qasm path:  training_samples/dj_indep_qiskit_7.qasm

Reading from .qasm path:  training_samples/dj_indep_qiskit_7.qasm

Reading from .qasm path:  training_samples/dj_indep_qiskit_7.qasm

Reading from .qasm path:  training_samples/dj_indep_qiskit_7.qasm

Reading from .qasm path:  training_samples/dj_indep_qiskit_7.qasm

Reading from .qasm path:  training_samples/dj_indep_qiskit_7.q